In [1]:
import numpy as np
import torch

print("Torch version:", torch.__version__)

Torch version: 1.7.1


In [2]:
! rm ./clip_classes/.DS_Store

rm: ./clip_classes/.DS_Store: No such file or directory


In [3]:
import os
#our the classes and images we want to test are stored in folders in the test set
class_names = os.listdir('./clip_classes/')
class_names.remove('_tokenization.txt')
class_names

['reading',
 'cooking',
 'dancing',
 'football',
 'sleeping',
 'swimming',
 'sitting',
 'jogging',
 'driving',
 'cycling']

In [4]:
#CLIP gets a lot better with the right prompting!
#be sure the tokenizations are in the same order as your class_names above!
%cat ./clip_classes/_tokenization.txt

A picture of a person reading
A picture of a person cooking
A picture of a person dancing
A picture of a person playing football
A picture of a person sleeping
A picture of a person swimming
A picture of a person sitting
A picture of a person jogging
A picture of a person driving
A picture of a person cycling


In [5]:
%%writefile ./clip_classes/_tokenization.txt
A picture of a person reading
A picture of a person cooking
A picture of a person dancing
A picture of a person playing football
A picture of a person sleeping
A picture of a person swimming
A picture of a person sitting
A picture of a person jogging
A picture of a person driving
A picture of a person cycling

Overwriting ./clip_classes/_tokenization.txt


In [6]:
candidate_captions = []
with open('./clip_classes/_tokenization.txt') as f:
    candidate_captions = f.read().splitlines()

In [7]:
import torch
import clip
from PIL import Image
import glob

def argmax(iterable):
    return max(enumerate(iterable), key=lambda x: x[1])[0]

device = "cuda" if torch.cuda.is_available() else "cpu"
model, transform = clip.load("ViT-B/32", device=device)

correct = []

#define our target classificaitons, you can should experiment with these strings of text as you see fit, though, make sure they are in the same order as your class names above
text = clip.tokenize(candidate_captions).to(device)

for cls in class_names:
    class_correct = []
    test_imgs = glob.glob('./clip_classes/' + cls + '/*.jpg')
    for img in test_imgs:
        print(img.split('/')[-1])
        image = transform(Image.open(img)).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text)
            
            logits_per_image, logits_per_text = model(image, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()

            pred = class_names[argmax(list(probs)[0])]
            print("Prediction Class : %s,  Label : %d,  Confidence : %.4f" % (pred, argmax(list(probs)[0]), probs[0][argmax(list(probs)[0])]*100))
            if pred == cls:
                correct.append(1)
                class_correct.append(1)
            else:
                correct.append(0)
                class_correct.append(0)
    
    print('\nAccuracy on class ' + cls + ' is : ' + str(sum(class_correct)/len(class_correct)*100) + "%\n\n")
print('\nOverall Accuracy is : ' + str(sum(correct)/len(correct)*100) + "%")

baby-reading-wearing-glasses-portrait-of-an-adorable-baby-sitting-up-wearing-eyeglasses-and-looking-stock-photo_csp5978912.jpg
Prediction Class : reading,  Label : 0,  Confidence : 91.7290
mother-reading-a-book-to-her-kid-stock-images_csp17473951.jpg
Prediction Class : reading,  Label : 0,  Confidence : 95.9057
family-reading-young-father-and-two-kids-reading-book-in-bed-stock-images_csp2686795.jpg
Prediction Class : reading,  Label : 0,  Confidence : 96.7331
baby-reading-wearing-glasses-portrait-of-an-adorable-baby-sitting-up-wearing-eyeglasses-and-looking-stock-photograph_csp5978889.jpg
Prediction Class : reading,  Label : 0,  Confidence : 90.9369
reading-book-young-woman-reading-book-stock-image_csp20576069.jpg
Prediction Class : reading,  Label : 0,  Confidence : 99.8553
story-time-with-grandpa-grandfather-reading-a-story-to-his-grandchild-stock-photo_csp3084272.jpg
Prediction Class : reading,  Label : 0,  Confidence : 91.4945
beautiful-woman-reading-a-book-picture_csp27799270.jpg


Prediction Class : cooking,  Label : 1,  Confidence : 99.0033
little-girl-puts-on-baking-cookies-little-cute-girl-puts-on-baking-cookies-stock-photo_csp17612145.jpg
Prediction Class : cooking,  Label : 1,  Confidence : 99.2886
cooking-an-african-woman-is-making-food-the-traditional-way-picture_csp10909275.jpg
Prediction Class : cooking,  Label : 1,  Confidence : 99.9682
cooking-two-little-girl-learning-how-to-cook-picture_csp2913671.jpg
Prediction Class : cooking,  Label : 1,  Confidence : 97.3713
cooking-girl-view-of-young-beautiful-girl-cooking-at-the-kitchen-pictures_csp30995282.jpg
Prediction Class : cooking,  Label : 1,  Confidence : 96.6936
happy-young-smiling-seller-in-a-cake-shop-stock-photo_csp28883849.jpg
Prediction Class : cooking,  Label : 1,  Confidence : 73.9707
woman-with-basket-young-girl-with-basket-in-bakery-stock-photograph_csp37190890.jpg
Prediction Class : cooking,  Label : 1,  Confidence : 94.5467
ready-to-eat-hands-of-male-chef-serving-vegetable-salad-on-plate-st

Prediction Class : dancing,  Label : 2,  Confidence : 81.1615
little-girl-dancing-happy-little-girl-dancing-isolated-on-white-background-stock-photo_csp11611243.jpg
Prediction Class : dancing,  Label : 2,  Confidence : 96.8075
young-woman-dancing-flamenco-on-black-stock-image_csp5768816.jpg
Prediction Class : dancing,  Label : 2,  Confidence : 90.8485
dancing-children-2-stock-images_csp2592565.jpg
Prediction Class : dancing,  Label : 2,  Confidence : 87.7477
two-adorable-ballet-students-two-little-ballet-students-practice-in-a-dance-studio-stock-image_csp9926646.jpg
Prediction Class : dancing,  Label : 2,  Confidence : 98.3118
little-girl-dancing-happy-little-girl-dancing-isolated-on-white-background-stock-photos_csp16758534.jpg
Prediction Class : dancing,  Label : 2,  Confidence : 81.1615
boy-dancing-happy-little-boy-dancing-isolated-on-white-background-stock-photograph_csp12021499.jpg
Prediction Class : dancing,  Label : 2,  Confidence : 87.6064
boy-dancing-happy-little-boy-dancing-i

Prediction Class : football,  Label : 3,  Confidence : 99.6358
football-action-play-4-teen-youth-football-in-action-on-field-during-game-stock-images_csp1070125.jpg
Prediction Class : football,  Label : 3,  Confidence : 99.9470
teen-youth-football-player-running-with-ball-during-game-stock-image_csp1069126.jpg
Prediction Class : football,  Label : 3,  Confidence : 99.7281

Accuracy on class football is : 100.0%


sleeping4.jpg
Prediction Class : sleeping,  Label : 4,  Confidence : 87.0762
sleeping5.jpg
Prediction Class : sleeping,  Label : 4,  Confidence : 99.7986
sleeping7.jpg
Prediction Class : sleeping,  Label : 4,  Confidence : 99.9434
sleeping6.jpg
Prediction Class : sleeping,  Label : 4,  Confidence : 99.7798
sleeping1.jpg
Prediction Class : sleeping,  Label : 4,  Confidence : 99.8159
sleeping8.jpg
Prediction Class : sleeping,  Label : 4,  Confidence : 99.1854
sleeping9.jpg
Prediction Class : sleeping,  Label : 4,  Confidence : 98.5633
sleeping10.jpg
Prediction Class : sleeping, 

Prediction Class : swimming,  Label : 5,  Confidence : 99.4181
portrait-of-people-swimming-floating-with-board-stock-photo_csp75873849.jpg
Prediction Class : swimming,  Label : 5,  Confidence : 99.5003
girl-swimming-backstroke-stock-image_csp5726546(1).jpg
Prediction Class : swimming,  Label : 5,  Confidence : 99.9129
baby-swimming-underwater-cute-little-baby-swimming-underwater-from-mother-to-father-in-a-pool-stock-photography_csp22941804.jpg
Prediction Class : swimming,  Label : 5,  Confidence : 99.9035
happy-mature-couple-in-the-swimming-pool-stock-photo_csp5652734.jpg
Prediction Class : swimming,  Label : 5,  Confidence : 99.3402
mother-and-baby-in-swimming-pool-happy-young-mother-and-her-little-son-adorable-laughing-baby-boy-stock-photograph_csp21894490.jpg
Prediction Class : swimming,  Label : 5,  Confidence : 99.9422
child-in-swimming-pool-happy-child-playing-in-swimming-pool-summer-vacations-concept-stock-photo_csp19165246.jpg
Prediction Class : swimming,  Label : 5,  Confidenc

Prediction Class : driving,  Label : 8,  Confidence : 94.7365
man-driving-a-car-inside-shoot-stock-image_csp14214363.jpg
Prediction Class : driving,  Label : 8,  Confidence : 99.6576
driving-a-car-man-at-the-wheel-the-car-stock-photo_csp5343902.jpg
Prediction Class : driving,  Label : 8,  Confidence : 99.5513
man-driving-car-man-driving-van-or-suv-stock-photo_csp5998262(1).jpg
Prediction Class : driving,  Label : 8,  Confidence : 99.7653
young-asian-woman-driver-driving-car-picture_csp27096413.jpg
Prediction Class : driving,  Label : 8,  Confidence : 99.9053
man-driving-a-car-stock-photography_csp0054090.jpg
Prediction Class : driving,  Label : 8,  Confidence : 97.8840
young-black-woman-driver-taking-a-rest-in-her-convertible-car-african-people-stock-image_csp15424068.jpg
Prediction Class : driving,  Label : 8,  Confidence : 97.2648
two-friends-driving-in-car-youth-lifestyle-two-smiling-friends-women-driving-in-car-stock-photo_csp2480844.jpg
Prediction Class : driving,  Label : 8,  Con

Prediction Class : cycling,  Label : 9,  Confidence : 99.8333

Accuracy on class cycling is : 100.0%



Overall Accuracy is : 99.69879518072288%
